# 📘 How to Use This Notebook
**Welcome!** If you are new to Jupyter Notebooks, here is a quick guide:

1.  **Cells**: This document is divided into blocks called "cells". Some are text (like this one) and some are code.
2.  **Running Code**: To run a code cell, click inside it and press the **Play Button (▶)** on the left side of the cell, or press `Shift + Enter` on your keyboard.
3.  **Order Matters**: Run the cells from top to bottom. The computer needs to "remember" the definitions in the first cells to make the later cells work.
4.  **Outputs**: The results of the code (like "Created Club: Badminton Smashers") will appear immediately below the cell you ran.

**Goal of this Notebook**: We are using this to *test* the logic for Tournaments (creating brackets, deciding winners) in simple Python code before we spend hours building the complex Mobile App screens. It's a sandbox! 🏖️

# Tournament Features Planning & Simulation

This notebook outlines the data models and logic for implementing the new Tournament features in SmashTracker.
It simulates the behavior of creating tournaments, recording matches, and viewing details (winner & match history).

In [ ]:
import uuid
from datetime import datetime

# 1. Define Data Structures

class Player:
    def __init__(self, name, id=None):
        self.id = id if id else str(uuid.uuid4())[:8]
        self.name = name

    def __repr__(self):
        return f"{self.name}"

class Match:
    def __init__(self, team1, team2, score1=0, score2=0):
        self.id = str(uuid.uuid4())[:8]
        self.team1 = team1 # List of Players
        self.team2 = team2 # List of Players
        self.score1 = score1
        self.score2 = score2
        self.is_completed = False
        self.winner = None

    def complete_match(self, score1, score2):
        self.score1 = score1
        self.score2 = score2
        self.is_completed = True
        if score1 > score2:
            self.winner = self.team1
        else:
            self.winner = self.team2

    def __repr__(self):
        t1_names = "/".join([p.name for p in self.team1])
        t2_names = "/".join([p.name for p in self.team2])
        status = f"[{self.score1}-{self.score2}]" if self.is_completed else "[Pending]"
        return f"{t1_names} vs {t2_names} {status}"

class Tournament:
    def __init__(self, name, club_id):
        self.id = str(uuid.uuid4())[:8]
        self.name = name
        self.club_id = club_id
        self.status = "Draft" # Draft, Active, Completed
        self.matches = []
        self.participants = []
        self.winner = None

    def add_participant(self, player):
        self.participants.append(player)
    
    def add_match(self, match):
        self.matches.append(match)

    def set_winner(self, team_or_player):
        self.winner = team_or_player
        self.status = "Completed"

class Club:
    def __init__(self, name):
        self.id = str(uuid.uuid4())[:8]
        self.name = name
        self.tournaments = []

    def create_tournament(self, name):
        t = Tournament(name, self.id)
        self.tournaments.append(t)
        return t

# --- Testing the Structures ---
club = Club("Badminton Smashers")
print(f"Created Club: {club.name}")

p1 = Player("Girish")
p2 = Player("Sandhya")
p3 = Player("John")
p4 = Player("Doe")

print("Players created.")

In [ ]:
# 2. Implement Tournament Logic (Simulating a Knockout Bracket)

def simulate_tournament_flow(tournament):
    print(f"--- Starting Tournament: {tournament.name} ---")
    tournament.status = "Active"
    
    # Round 1: Semi-Finals
    m1 = Match([p1], [p2]) # Singles for simplicity
    m2 = Match([p3], [p4])
    
    tournament.add_match(m1)
    tournament.add_match(m2)
    
    # Simulate playing matches
    print("Playing Semi-Finals...")
    m1.complete_match(21, 15) # p1 wins
    print(f"Match 1 Result: {m1}")
    
    m2.complete_match(10, 21) # p4 wins
    print(f"Match 2 Result: {m2}")
    
    # Final
    print("Playing Final...")
    final_winner1 = m1.winner
    final_winner2 = m2.winner
    
    final_match = Match(final_winner1, final_winner2)
    tournament.add_match(final_match)
    
    final_match.complete_match(19, 21) # p4 wins total
    print(f"Final Result: {final_match}")
    
    tournament.set_winner(final_match.winner)
    print(f"--- Tournament Ended. Winner: {tournament.winner[0].name} ---")

# Run Simulation
t1 = club.create_tournament("Spring Championship 2026")
t1.add_participant(p1)
t1.add_participant(p2)
t1.add_participant(p3)
t1.add_participant(p4)

simulate_tournament_flow(t1)

In [ ]:
# 3. Display Tournament Details (The Click Handler Logic)

def get_tournament_details(tournament_id, club):
    # Find tournament
    tournament = next((t for t in club.tournaments if t.id == tournament_id), None)
    
    if not tournament:
        return "Tournament not found"
        
    print(f"=== {tournament.name} Details ===")
    print(f"Status: {tournament.status}")
    
    if tournament.winner:
        # Assuming winner is a list of Players (Team)
        winner_names = ", ".join([p.name for p in tournament.winner])
        print(f"🏆 WINNER: {winner_names} 🏆")
    else:
        print("Winner: TBD")
        
    print("\n--- Match History ---")
    for i, m in enumerate(tournament.matches):
        print(f"{i+1}. {m}")
    print("=================================")

# Test the display function
get_tournament_details(t1.id, club)

In [ ]:
# 4. Simulation with New Data (Scenario: Doubles League)

print("\n\n--- Scenario 2: Doubles League ---")
t2 = club.create_tournament("Doubles Mixer")

# Create new players
p5, p6, p7, p8 = Player("Alice"), Player("Bob"), Player("Charlie"), Player("Delta")

# Create Teams (Logic would be more complex in real app)
team_A = [p5, p6]
team_B = [p7, p8]

league_match = Match(team_A, team_B)
t2.add_match(league_match)

# Simulate match played
league_match.complete_match(21, 19)
t2.set_winner(league_match.winner) # In a 1-match tournament

# Display Details
get_tournament_details(t2.id, club)

In [ ]:
# 5. Interactive Visual Bracket Logic (Tree Structure)

def generate_bracket_tree(tournament):
    """
    Parses a tournament's match history to build a visual tree structure.
    In a real app, this JSON would be passed to a frontend component.
    """
    if not tournament.matches:
        return "No matches to display."

    # Identify Rounds
    # Logic: If 4 players -> 2 Semi-Finals (Round 1) -> 1 Final (Round 2)
    # This is a simplified logic assuming matches were added in order
    
    bracket_data = {
        "tournament_name": tournament.name,
        "rounds": []
    }
    
    # Heuristic: Group matches by rounds based on participant count or creation order
    # For this simulation, we know matches 0&1 are Round 1, Match 2 is Round 2
    
    rounds = {} # Key: Round Number, Value: List of Match Details
    
    for i, match in enumerate(tournament.matches):
        team1_name = " / ".join([p.name for p in match.team1])
        team2_name = " / ".join([p.name for p in match.team2])
        winner_name = " / ".join([p.name for p in match.winner]) if match.winner else "TBD"
        
        match_node = {
            "match_id": match.id,
            "team1": team1_name,
            "team2": team2_name,
            "score": f"{match.score1} - {match.score2}",
            "winner": winner_name,
            "status": "Completed" if match.is_completed else "Pending"
        }
        
        # Simplified Round Assignment for Demo
        # In a real system, 'round' would be a property of the Match object
        round_num = 1 if i < 2 else 2 
        
        if round_num not in rounds:
            rounds[round_num] = []
        rounds[round_num].append(match_node)

    # Format for Display
    for r_num in sorted(rounds.keys()):
        bracket_data["rounds"].append({
            "round_number": r_num,
            "round_name": "Semi-Finals" if r_num == 1 else "Finals",
            "matches": rounds[r_num]
        })
        
    return bracket_data

# Display the Visual Tree Structure
def print_visual_bracket(bracket_data):
    print(f"\n🏟️  BRACKET: {bracket_data['tournament_name']} 🏟️\n")
    
    for round_info in bracket_data['rounds']:
        print(f"--- {round_info['round_name']} ---")
        for m in round_info['matches']:
            t1 = m['team1'].ljust(15)
            t2 = m['team2'].ljust(15)
            score = m['score']
            print(f"  ┌── {t1}")
            print(f"  │vs ({score})")
            print(f"  └── {t2}")
            print(f"      => Winner: {m['winner']}\n")
        print("         |\n         V\n")

# Run on the first tournament (t1) which had a full tree
bracket_json = generate_bracket_tree(t1)
print_visual_bracket(bracket_json)